<a href="https://colab.research.google.com/github/siddharthsankhe/sid/blob/master/text_summarisation1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download('punkt')

import bs4 as bs
import urllib.request
import re

nltk.download('stopwords')
!python -m spacy download en_core_web_lg


In [0]:
import spacy
nlp = spacy.load('en_core_web_lg')
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/The_Avengers_(2012_film)').read()
soup = bs.BeautifulSoup(source,'lxml')
text=""
#for paragraph in soup.find_all('p'):
for paragraph in soup.find_all('p'):
  text += paragraph.text

In [66]:
text

'\nMarvel\'s The Avengers[6] (classified under the name Marvel Avengers Assemble in the United Kingdom and Ireland),[3][7] or simply The Avengers, is a 2012 American superhero film based on the Marvel Comics superhero team of the same name, produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures.[N 1] It is the sixth film in the Marvel Cinematic Universe (MCU). The film was written and directed by Joss Whedon and features an ensemble cast that includes Robert Downey Jr., Chris Evans, Mark Ruffalo, Chris Hemsworth, Scarlett Johansson, and Jeremy Renner as the titular Avengers team, alongside Tom Hiddleston, Clark Gregg, Cobie Smulders, Stellan Skarsgård, and Samuel L. Jackson. In the film, Nick Fury, director of the spy agency S.H.I.E.L.D., recruits Tony Stark, Steve Rogers, Bruce Banner, and Thor to form a team that must stop Thor\'s brother Loki from subjugating Earth.\nThe film\'s development began when Marvel Studios received a loan from Merrill Lynch in Ap

In [0]:
text = re.sub(r'\[[0-9]*\]',' ',text)
text = re.sub(r'\s+',' ',text)
clean_text = text.lower()
#clean_text = re.sub(r'\W',' ',clean_text)
clean_text = re.sub(r'\d',' ',clean_text)
clean_text = re.sub(r'\s+',' ',clean_text)
clean_text = re.sub('<[^>]+>', '', clean_text)

In [68]:
clean_text

' marvel\'s the avengers (classified under the name marvel avengers assemble in the united kingdom and ireland), or simply the avengers, is a american superhero film based on the marvel comics superhero team of the same name, produced by marvel studios and distributed by walt disney studios motion pictures.[n ] it is the sixth film in the marvel cinematic universe (mcu). the film was written and directed by joss whedon and features an ensemble cast that includes robert downey jr., chris evans, mark ruffalo, chris hemsworth, scarlett johansson, and jeremy renner as the titular avengers team, alongside tom hiddleston, clark gregg, cobie smulders, stellan skarsgård, and samuel l. jackson. in the film, nick fury, director of the spy agency s.h.i.e.l.d., recruits tony stark, steve rogers, bruce banner, and thor to form a team that must stop thor\'s brother loki from subjugating earth. the film\'s development began when marvel studios received a loan from merrill lynch in april . after the s

In [0]:
doc = nlp(clean_text, disable=['parser', 'ner'])
 

In [0]:
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
stopwords = nltk.corpus.stopwords.words('english')
tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]

In [0]:
tokens = ' '.join(tokens)

In [72]:
tokens

"marvel avenger classify name marvel avenger assemble united kingdom ireland simply avenger american superhero film base marvel comic superhero team name produce marvel studio distribute walt disney studio motion pictures.[n sixth film marvel cinematic universe mcu . film write direct joss whedon feature ensemble cast include robert downey jr . chris evans mark ruffalo chris hemsworth scarlett johansson jeremy renner titular avenger team alongside tom hiddleston clark gregg cobie smulder stellan skarsgård samuel l. jackson . film nick fury director spy agency s.h.i.e.l.d . recruit tony stark steve rogers bruce banner thor form team must stop thor 's brother loki subjugate earth . film 's development begin marvel studio receive loan merrill lynch april . success film iron man may marvel announce avenger would release july . signing johansson march film push back release . whedon bring board april rewrite original screenplay zak penn . production begin april albuquerque new mexico move c

In [0]:
sent_tokenize=nltk.sent_tokenize(tokens)

In [76]:
sent_tokenize[:2]

['marvel avenger classify name marvel avenger assemble united kingdom ireland simply avenger american superhero film base marvel comic superhero team name produce marvel studio distribute walt disney studio motion pictures.',
 '[n sixth film marvel cinematic universe mcu .']

In [0]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(sent_tokenize)

In [0]:
feature_names=vectorizer.get_feature_names()


In [109]:
import numpy as np
feature_names=np.asarray(feature_names)
feature_names.shape

(1841,)

In [110]:
X.toarray().shape

(401, 1841)

In [0]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

sorted_items=sort_coo(X.tocoo())

In [117]:
sorted_items[:2]

[(1447, 1.0), (1040, 1.0)]

In [0]:

def extract_topn_from_vector(feature_names, sorted_items):

    
 
    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:

        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 

    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results
feature_names=vectorizer.get_feature_names()
keywords=extract_topn_from_vector(feature_names,sorted_items)


In [0]:
sent2score = {}

In [144]:
sent_tokenize[:2]

['marvel avenger classify name marvel avenger assemble united kingdom ireland simply avenger american superhero film base marvel comic superhero team name produce marvel studio distribute walt disney studio motion pictures.',
 '[n sixth film marvel cinematic universe mcu .']

In [0]:
for sentence in text.split('.'):
    for word in nltk.word_tokenize(sentence.lower()):
        if word in keywords.keys():
            if len(sentence.split(' ')) < 25:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = keywords[word]
                else:
                    sent2score[sentence] += keywords[word]

In [146]:
                    
import heapq
best_sentences = heapq.nlargest(5, sent2score, key=sent2score.get)

print('---------------------------------------------------------')
for sentence in best_sentences:
    print(sentence)

---------------------------------------------------------
 Filming also took place in the large vacuum chamber at the NASA Plum Brook Station near Sandusky, Ohio
 Harry Dean Stanton cameos as a security guard, and Polish film director Jerzy Skolimowski appears as Georgi Luchkov, Romanoff's interrogator
" Silvestri developed the score with the London Symphony Orchestra at Abbey Road Studios in London, England
, and his lieutenant Agent Maria Hill arrive at a remote research facility during an evacuation, where physicist Dr
 Meanwhile, Fury's superiors from the World Security Council attempt to end the invasion by launching a nuclear missile at Midtown Manhattan
